In [2]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, roc_auc_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.initializers import GlorotUniform

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-30 16:15:21.464138: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

labels = pd.read_csv('labels.csv')

In [31]:
# Find bounding boxes of all 4 image sizes
def find_bounding_box(img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find all non-black pixels
    coords = cv2.findNonZero(gray)
    
    # Find the bounding box of those pixels
    x, y, w, h = cv2.boundingRect(coords)
    print(x, y, w, h)

# Define the paths to your images
image_paths = [
    'images/Line/AI&Robotics_2019-01-15 00:00:00_14.png',
    'images/Line/AI&Robotics_2019-01-24 00:00:00_30.png',
    'images/Line/AI&Robotics_2019-03-28 00:00:00_90.png',
    'images/Line/AI&Robotics_2019-06-20 00:00:00_180.png', 
    'images/Line/AI&Robotics_2019-12-26 00:00:00_365.png'   
]

# Crop each image and save the result
for path in image_paths:
    cropped_image = find_bounding_box(path)

80 100 85 120
80 100 132 120
80 100 226 120
80 100 414 120
80 100 602 120


In [5]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
        elif number == 365:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+602]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [6]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_31251/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [7]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [8]:
# Sort the DataFrame by date
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

                                                  Image  TimePrediction  \
0     images/OHLC/S&P500_2019-02-01 00:00:00_14_crop...               5   
1149  images/OHLC/DAX_2019-02-01 00:00:00_180_croppe...              90   
6531  images/Line/EmergingMarkets_2019-02-01 00:00:0...              30   
6530  images/Line/EmergingMarkets_2019-02-01 00:00:0...               5   
6500  images/Line/EmergingMarkets_2019-02-01 00:00:0...               5   

       LastPrice  FuturePrice  Label  \
0     270.059998   270.140015      1   
1149   25.042000    27.209999      1   
6531   42.759998    42.549999      0   
6530   42.759998    42.400002      0   
6500   42.759998    42.400002      0   

                                            Image_Array       Date  
0     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
1149  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
6531  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13... 2019-02-01  
6530  [[0, 0, 0, 0, 0, 0, 0, 0, 

In [9]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

1    6620
0    3780
Name: Label, dtype: int64


In [39]:
'''def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Set dilation rate and strides
        strides = (1, 3)

        # Add Convolutional layer
        model.add(Conv2D(filters, kernel_size=(5, 3), strides=strides, padding='same',
                         input_shape=input_shape if i == 0 else model.output_shape[1:]))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer only if dimensions allow
        if model.output_shape[1] >= 2 and model.output_shape[2] >= 2:
            model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add Flatten and Dense layers for final prediction
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax')) 

    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model'''

In [16]:
def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    # Determine the number of blocks based on the input image size
    if num_blocks == 2:
        dilation_rate = (1, 1)
    elif num_blocks == 3:
        dilation_rate = (1, 2)
    else:  # Assuming 60-day images
        dilation_rate = (1, 3)

    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Add Convolutional layer
        if i == 0:  # Apply specific strides and dilation rate only for the first layer
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 3), padding='same', 
                             input_shape=input_shape if i == 0 else None))
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 1), dilation_rate=dilation_rate, padding='same', 
                             input_shape=input_shape if i == 0 else None))
        else:
            model.add(Conv2D(filters, kernel_size=(5, 3), padding='same'))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer
        model.add(MaxPooling2D(pool_size=(2, 1)))

    # Add Flatten and Dense layers for final prediction
    dense_units = {2: 64, 3: 128, 4: 264}.get(num_blocks, 64) 
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# Train and evaluate the model for each img_type, timeframe, and prediction

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:
                print(f"Predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")
                
                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
                
                # Create and compile your CNN model based on timeframe
                if timeframe == 14:
                    model = create_cnn_model(2, input_shape=(120, 85, 1))
                elif timeframe == 30:
                    model = create_cnn_model(3, input_shape=(120, 132, 1))
                elif timeframe == 90:
                    model = create_cnn_model(3, input_shape=(120, 226, 1))
                elif timeframe == 180:
                    model = create_cnn_model(4, input_shape=(120, 414, 1))
                elif timeframe == 365:
                    model = create_cnn_model(4, input_shape=(120, 602, 1))
                
                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = np.array(data['LastPrice'].tolist())
                futurePrice = np.array(data['FuturePrice'].tolist())
                y = data['Label']
                # Split the data into training and validation sets in temporal order
                split_index = int(len(X) * 0.7)
                X_train, X_test = X[:split_index], X[split_index:]
                y_train, y_test = y[:split_index], y[split_index:]
                lastPrice_test = lastPrice[split_index:]
                futurePrice_test = futurePrice[split_index:]

                # Shuffle test data
                X_test, y_test, lastPrice_test, futurePrice_test = shuffle(X_test, y_test, lastPrice_test, futurePrice_test, random_state=0)
                
                # Train the model on 10 windows of data
                window_size = len(X_train)/10
                num_windows = 10

                for epoch in range(10):  # 10 epochs
                    start = 0
                    end = int(window_size * (epoch + 1))
                    X_temp = X_train[start:end]
                    y_temp = y_train[start:end]
                    split_index = int(len(X) * 0.9)
                    X_train_temp, X_val_temp = X_temp[:split_index], X_temp[split_index:]
                    y_train_temp, y_val_temp = y_temp[:split_index], y_temp[split_index:]
                    model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_data=(X_val_temp, y_val_temp))            
                
                # Evaluate the model on test data
                y_pred = model.predict(X_test)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)
                
                accuracy = accuracy_score(y_test, y_pred_binary)
                precision = precision_score(y_test, y_pred_binary)
                recall = recall_score(y_test, y_pred_binary)
                f1_score = fbeta_score(y_test, y_pred_binary, beta=1)

                y_test_array = y_test.values.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                # Calculate Rate of Return RoR
                ror = np.where(y_pred_binary == 1, ((futurePrice_test - lastPrice_test)/lastPrice_test), 0)
                # Drop all 0s from the array
                ror = ror[ror != 0]
                # Calculate average RoR
                avg_ror = np.mean(ror)
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1_score}")
                print(f"Hit Rate: {hit_rate}")
                print(f"Average RoR: {avg_ror}")
                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Average_RoR': avg_ror
                }, ignore_index=True)
                
                # Save the model
                model.save(f"models/{img_type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as {img_type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'evaluation_scores.csv'.")

In [ ]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
            
            # Create and compile your CNN model based on timeframe
            if timeframe == 14:
                model = create_cnn_model(2, input_shape=(120, 85, 1))
            elif timeframe == 30:
                model = create_cnn_model(3, input_shape=(120, 132, 1))
            elif timeframe == 90:
                model = create_cnn_model(3, input_shape=(120, 226, 1))
            elif timeframe == 180:
                model = create_cnn_model(4, input_shape=(120, 414, 1))
            elif timeframe == 365:
                model = create_cnn_model(4, input_shape=(120, 602, 1))
            
            X = np.array(data['Image_Array'].tolist()) / 255.0
            lastPrice = np.array(data['LastPrice'].tolist())
            futurePrice = np.array(data['FuturePrice'].tolist())
            y = data['Label']
            # Split the data into training and validation sets in temporal order
            split_index = int(len(X) * 0.7)
            X_train, X_test = X[:split_index], X[split_index:]
            y_train, y_test = y[:split_index], y[split_index:]
            lastPrice_test = lastPrice[split_index:]
            futurePrice_test = futurePrice[split_index:]
            
            window_size = len(X_train)/10
            num_windows = 10

            for epoch in range(10):  # 10 epochs
                start = 0
                end = int(window_size * (epoch + 1))
                X_temp = X_train[start:end]
                y_temp = y_train[start:end]
                split_index = int(len(X) * 0.9)
                X_train_temp, X_val_temp = X_temp[:split_index], X_temp[split_index:]
                y_train_temp, y_val_temp = y_temp[:split_index], y_temp[split_index:]
                model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=1, validation_data=(X_val_temp, y_val_temp))            
            
            # Evaluate the model
            y_pred = model.predict(X_test)
            # Convert predictions to binary: if > 0.5 then 1 else 0
            y_pred_binary = np.where(y_pred > 0.5, 1, 0)
            
            accuracy = accuracy_score(y_test, y_pred_binary)
            precision = precision_score(y_test, y_pred_binary)
            recall = recall_score(y_test, y_pred_binary)
            f1_score = fbeta_score(y_test, y_pred_binary, beta=1)

            y_test_array = y_test.values.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
            correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
            hit_rate = correct_predictions / len(y_test_array)

            # Calculate Rate of Return RoR
            ror = np.where(y_pred_binary == 1, ((futurePrice_test - lastPrice_test)/lastPrice_test), 0)
            # Drop all 0s from the array
            ror = ror[ror != 0]
            # Calculate average RoR
            avg_ror = np.mean(ror)
            
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1_score}")
            print(f"Hit Rate: {hit_rate}")
            print(f"Average RoR: {avg_ror}")
            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Average_RoR': avg_ror
            }, ignore_index=True)
            
            # Save the model
            model.save(f"models/combined_{timeframe}_{prediction}.h5")
            print(f"Model saved as combined_{timeframe}_{prediction}.h5")
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('combined_model_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'combined_model_evaluation_scores.csv'.")